In [1]:
import numpy as np 
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, BatchNormalization, Conv2DTranspose, Concatenate, Reshape, LeakyReLU, Input, UpSampling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.compat.v1 import ConfigProto, InteractiveSession

In [2]:
config=ConfigProto
config.gpu_options.allow_growth=True
sess=InteractiveSession(config=config)

AttributeError: 'google.protobuf.pyext._message.FieldProperty' object has no attribute 'allow_growth'

In [3]:
(x_train, y_train), (x_test, y_test)=mnist.load_data()

In [4]:
x_train=x_train.reshape((-1,28,28,1))-127.5/127.5
x_test=x_test.reshape((-1,28,28,1))-127.5/127.5
ohe=OneHotEncoder(sparse=False)
y_train=ohe.fit_transform(y_train.reshape((-1,1)))
y_test=ohe.transform(y_test.reshape((-1,1)))

In [5]:
def gen(latent):
    in1=Input(shape=(latent,1))
    x1=Dense(7*7*128)(in1)
    x1=LeakyReLU(alpha=0.2)(x1)
    x1=Reshape((7,7,128))(x1)

    in2=Input(shape=(10))
    x2=Dense(49)(in2)
    x2=LeakyReLU(alpha=0.2)(x2)
    x2=Reshape((7,7,1))(x2)

    x=Concatenate()([x1,x2])

    x=UpSampling2D()(x)
    x=Conv2DTranspose(128, (3,3), padding='same')(x)
    x=LeakyReLU(alpha=0.2)(x)

    x=UpSampling2D()(x)
    x=Conv2DTranspose(64, (3,3), padding='same')(x)
    x=LeakyReLU(alpha=0.2)(x)

    x=Conv2DTranspose(1, (3,3), padding='same', activation='tanh')(x)

    return Model([in1, in2], x)

In [6]:
def disc():
    in1=Input(shape=(28,28,1))

    in2=Input(shape=(10))
    x2=Dense(28*28)(in2)
    x2=LeakyReLU(alpha=0.2)(x2)
    x2=Reshape((28,28,1))(x2)

    x=Concatenate()([in1,x2])

    x=Conv2D(64, 3, padding='same')(x)
    x=MaxPool2D()(x)
    x=LeakyReLU(alpha=0.2)(x)

    x=Conv2D(128, 3, padding='same')(x)
    x=MaxPool2D()(x)
    x=LeakyReLU(alpha=0.2)(x)

    x=Flatten()(x)
    x=Dense(1, activation='sigmoid')(x)
    
    return Model([in1, in2], x)

In [7]:
def models(latent):
    d=disc()
    g=gen(latent)
    
    d.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='categorical_crossentropy', metrics=['accuracy'])

    d.trainable=False
    adv=Model(g.input, d([g.output,g.input[1]]))

    adv.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')

    return g, d, adv

In [8]:
def train(latent, epochs, batch_size, x, y, ohe, shuffle, save_interval, image_gen):
    assert x.shape[0]==y.shape[0], "x.shape[0]!=y.shape[0]"
    assert batch_size>1, "Batch size should be more than 1" 
    g, d, a=models(latent)
    n=y.shape[0]
    l=np.arange(0,n)
    hist={
        "discriminator_loss":list(),
        "discriminator_accuracy":list(),
        "adversarial_loss":list(),
    }
    for i in range(epochs):
        if shuffle:
            l=np.random.shuffle(l)
        tld=list()
        tad=list()
        tla=list()
        for j in range(0, n-batch_size+1, batch_size):
            f=j+batch_size
            if f>=n:
                f=n
            
            latentnoise=np.random.random((batch_size//2, latent))
            randlabel=np.random.randint(10, size=batch_size//2).reshape((-1,1))
            randlabel=ohe.transform(randlabel)

            fpred=g.predict([latentnoise, randlabel])

            dlossf, daccf=d.train_on_batch([fpred, randlabel], np.zeros(batch_size//2))

            realx=x[l[i:j+1-batch_size//2],:,:,:]
            realy=y[l[i:j+1-batch_size//2],:]
            latentnoise=np.random.random((batch_size//2, latent))

            dlossr, daccr=d.train_on_batch([realx, realy], np.ones(batch_size//2))

            dloss=0.5*(dlossf+dlossr)
            dacc=0.5*(daccf+daccr)

            latentnoise=np.random.random((batch_size, latent))
            randlabel=np.random.randint(10, size=batch_size).reshape((-1,1))
            aloss=a.train_on_batch([latentnoise, randlabel], np.ones(batch_size))

            tld.append(dloss)
            tla.append(aloss)
            tad.append(dacc)
            bar="="*((j*9)//n)+">"+"-"*(9-((j*9)//n))
            print("Epoch %d/%d [%s]: d_loss:%f d_acc:%f a_loss:%f"%(i+1,epochs,bar,dloss,dacc,aloss), end='\r')
        print("Epoch %d/%d: d_loss:%f d_acc:%f a_loss:%f"%(i+1,epochs,dloss,dacc,aloss))
        hist["discriminator_loss"].append(tld)
        hist["discriminator_accuracy"].append(tad)
        hist["adversarial_loss"].append(tla)
        if (i+1)%save_interval==0:
            g.save("models/gen%d.h5"%i)
            a.save("models/adv%d.h5"%i)
            d.save("models/dis%d.h5"%i)
        if (i+1)%image_gen==0:
            fig=plt.figure()
            latentnoise=np.random.random((10, latent))
            lab=np.arange(10).reshape((-1,1))
            lab=ohe.transform(lab)
            im=g.predict([latentnoise, lab])
            for i in range(10):
                ax=fig.add_subplot(5, 2, i+1)
                ax.imshow(im[i])
                ax.axis('off')
            plt.savefig('images/%d.jpg'%i+1)~~
    return hist, g, d, a

In [9]:
h,_,_,_=train(100, 10, 32, x_train, y_train, ohe, True, 1, 1)

InvalidArgumentError:  Input to reshape is a tensor with 10035200 values, but the requested shape has 100352
	 [[node model_1/reshape_1/Reshape (defined at <ipython-input-8-4916bce3e0e2>:27) ]] [Op:__inference_predict_function_709]

Function call stack:
predict_function
